In [28]:
import sys  
sys.path.insert(0, '../../')
import RadonDF_Handler
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from statsmodels.tsa.api import VAR, VARMAX
plt.style.use('fivethirtyeight')

In [38]:
D003_df = pd.read_csv('./../../../../../Data/SensorsData/interpolated_D003_data.csv')
D003_df = D003_df.iloc[913:]
D003_df['time'] =  pd.to_datetime(D003_df['time'], format='%Y-%m-%d %H:%M:%S')
D003_df.index = D003_df['time']
D003_df = D003_df.drop(['time','H','P','CO2'],axis = 1)
D003_df 

,T,Rn
time,,
2019-09-19 14:00:00,32.216000,449.698000
2019-09-19 15:00:00,32.783333,173.120000
2019-09-19 16:00:00,32.661667,43.223333
2019-09-19 17:00:00,33.540000,43.876000
2019-09-19 18:00:00,32.928333,122.895000
...,...,...
2021-12-31 19:00:00,28.650000,956.366667
2021-12-31 20:00:00,28.758000,1275.534000
2021-12-31 21:00:00,28.730000,1321.585000


In [39]:
def plotter(df,x,y,title):
    fig = px.line(df, x=x, y=y, title=title)

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(step="all")
            ])
        )
    )
    fig.show()

In [40]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)
    for key,val in result[4].items():
        out[f'critical value ({key})']=val
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
# columns = [np.log(D003_df['T'].values),np.log(D003_df['Rn'].values),np.log(D003_df['H'].values),np.log(D003_df['P'].values),np.log(D003_df['CO2'].values)]
adf_test( D003_df['T'], title='Temperature')
print('\n-----------------------------------')

adf_test(D003_df['Rn'], title='Radon')
print('\n-----------------------------------')

# adf_test(D003_df['H'], title='Humidity')
# print('\n-----------------------------------')

# adf_test(D003_df['P'], title='Pressure')
# print('\n-----------------------------------')

# adf_test(D003_df['CO2'], title='CO2')

In [ ]:
# The correlation heatmap is based on the Pearson Correlation coefficient,
# which states that any value lower than 0.8 or greater than -0.8 is insignificant,
# as it is the most commonly used coefficient in statistics.
D003_df.corr(min_periods=7)

In [ ]:
train_data, test_data = train_test_split(D003_df, test_size=0.2, shuffle=False)

In [ ]:
print('train data: ',train_data.shape)
print('test data: ',test_data.shape)

In [ ]:
model = VAR(train_data)

In [ ]:
sorted_order = model.select_order(maxlags=30)
print(sorted_order.summary())

In [ ]:
var_model = VARMAX(train_data,order=(10,0),enforce_stationarity=True)


In [42]:
var_model

In [45]:
%%time

fitted_model = var_model.fit(disp=True,full_output=True)

CPU times: total: 1h 5min 54s
Wall time: 47min 49s


In [46]:
fitted_model.summary()

Dep. Variable:,"['T', 'Rn', 'H', 'P', 'CO2']",No. Observations:,15288
Model:,VAR(10),Log Likelihood,-149307.428
,+ intercept,AIC,299154.857
Date:,"Fri, 06 Jan 2023",BIC,301216.259
Time:,10:10:20,HQIC,299838.164
Sample:,0,,
,- 15288,,
Covariance Type:,opg,,
Ljung-Box (L1) (Q):,"0.10, 0.00, 0.01, 0.00, 0.00",Jarque-Bera (JB):,"1639565.82, 38490.08, 4121587.52, 59945.56, 235817.61"
Prob(Q):,"0.76, 0.98, 0.93, 0.98, 1.00",Prob(JB):,"0.00, 0.00, 0.00, 0.00, 0.00"
Heteroskedasticity (H):,"1.11, 1.57, 1.23, 0.68, 2.63",Skew:,"-2.63, -0.11, 0.41, 0.28, 2.32"


In [53]:
n_forecast = 1
predict = fitted_model.predict(test_data)
predict
# predictions=predict.predicted_mean
     

KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

In [ ]:
predict

In [ ]:
# result = model.fit(10)
# result.summary()

In [ ]:
# lagged_Values = train_data.values[-10:]


In [ ]:
# lagged_Values

In [ ]:
# pred = result.forecast(y=lagged_Values, steps=12) 

# idx = pd.date_range('2015-01-01', periods=12, freq='MS')
# df_forecast=pd.DataFrame(data=pred, index=idx, columns=['money_2d', 'spending_2d'])